In [8]:
! pip install dash-dangerously-set-inner-html

In [9]:
# Global import

import datetime as dt

import pandas as pd
import plotly.graph_objects as go

from dash_mdc_neptune import themes

In [2]:
# Global variables

TODAY = pd.to_datetime(dt.date.today(), utc=True)

CURVE_UID = "PWRTE"
PUBLISHED_AT = TODAY - pd.tseries.offsets.BDay(10)

# PAGE 1

In [4]:
# Extract MA fixings

st_regex = r"^(D-)|(Week-)"

query_params = {
    "asset": CURVE_UID,
    "published_at": PUBLISHED_AT,
}

df_ma = nptc.retrieve_forward_fixings(**query_params)

df_ma = df_ma[df_ma["calendar_delivery"].str.match(st_regex) == True].copy()
df_ma["publication_date"] = df_ma["published_at"].dt.date

In [5]:
# Retrieve curve

query_params = {
    "curve_uid": CURVE_UID,
    "published_at": PUBLISHED_AT,
}

curve = nptc.retrieve_model(**query_params)

npta.curve_update_with_fixings(curve=curve, fixings=df_ma, created_at=PUBLISHED_AT)

In [6]:
# Retrieve ALL fixings

ts_list = []

for row in df_ma.itertuples():
    symbol = npta.symbol_from_dict({
        "contract": "FWD",
        "asset": row.asset,
        "intraday_delivery": row.intraday_delivery,
        "calendar_delivery": row.calendar_delivery,
    })
    query_params = {
            "curve_uid": symbol,
            "curve_type": "FIXING",
            "published_at": PUBLISHED_AT,
        }
    try:
        ts_tmp = nptc.retrieve_time_series(**query_params)
        ts_list.append(ts_tmp.rename(symbol))
    except Exception:
        pass

df_past = pd.concat(ts_list, join="outer", axis=1)

In [10]:
# Fixing colours

noos_palette = themes.NOOS_PALETTE_SEQUENTIAL
fixing_colour = {}

colour_counter = 0
for delivery in df_ma["calendar_delivery"].unique():
    fixing_colour[delivery] = noos_palette[colour_counter]
    colour_counter = (colour_counter + 1) %len(noos_palette)

## SECTION 1

### CARD 1

In [11]:
# Format fixings

df_latest = df_ma[["intraday_delivery", "calendar_delivery", "price", "publication_date"]]
df_latest = df_latest.sort_values(["calendar_delivery", "intraday_delivery"])
df_latest = df_latest.rename(
    columns={
        "intraday_delivery": "intraday delivery",
        "calendar_delivery": "calendar delivery",
        "price": "EUR/MWH",
        "publication_date": "publication date",
    }
)

In [12]:
# Format table

def colour_selector(row, colour_mapping=fixing_colour):
    colour_hex = colour_mapping[row["calendar delivery"]]
    return {col: f"background-color: {colour_hex}" for col in row.index}

latest_styler = df_latest.style
latest_styler = latest_styler.hide_index()
latest_styler = latest_styler.format({"EUR/MWH": "{:,.2f}"})
latest_styler = latest_styler.set_table_attributes('style="font-size: 0.8rem"')
latest_styler = latest_styler.apply(lambda x: colour_selector(x), axis=1)

In [ ]:
# Display table

# latest_styler

In [ ]:
# Display source

# print(
#     f"Source: https://api.noos.energy/v1"
#     f"/products/{CURVE_UID}"
#     f"?published_at={PUBLISHED_AT.tz_localize(None).isoformat()}"
# )

## SECTION 2

In [13]:
# Shared scatter configs

GO_LAYOUT = go.Layout(
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text='€/MWh',
        )
    ),
    template="noos_watermark+noos_base+noos_colorscale+min_modebar+hoover_xunified",
)

### CARD 1

In [14]:
# Extract volumes, prices

df_fixings = curve.fixings.to_pandas()

df_volumes = npta.domain.products.collections.IndexSet.from_pandas(df_fixings).volumes()
ts_prices = pd.to_numeric(df_fixings["price"])

In [15]:
# Format curves

df_wide = pd.DataFrame(
    data=2 * df_volumes.values * ts_prices.values,
    index=df_volumes.index,
    columns=df_volumes.columns
)
df_wide["PFC"] = curve.to_pandas()
df_wide = df_wide.tz_convert("Europe/Paris")

# Transform table from wide-to-long

df_long = df_wide.unstack().reset_index()
df_long = df_long.rename(columns={"level_0": "product", "level_1": "delivery_from", 0: "price"})
df_long["curve_type"] = df_long["product"].apply(lambda x: "forwards" if x == "PFC" else "products")

# Remove 0 points

df_long = df_long[~(df_long["price"] == 0)]

In [16]:
# Format graph 

data = []
for product, df_product in df_long.groupby("product"):
    if product == "PFC":
        data.append(
            go.Scatter(
                x=df_product["delivery_from"],
                y=df_product["price"],
                line_color=themes.NOOS_HTML_COLORS["primary"],
                name= "half-hourly forwards",
            )
        )

    else:
        symbol = npta.symbol_to_dict(product)
        if symbol['intraday_delivery'] == 'BASE':
            line_dash='solid'
        else:
            line_dash='dot'
        data.append(    
            go.Scatter(
                x=df_product["delivery_from"],
                y=df_product["price"],
                line_dash=line_dash,
                line_color=fixing_colour[symbol['calendar_delivery']],
                name=f"{symbol['intraday_delivery']} {symbol['calendar_delivery']}",
                legendgroup=f"{symbol['intraday_delivery'].lower()} products",
                legendgrouptitle_text=f"{symbol['intraday_delivery'].lower()} products",
            )
        )

fig_1 = go.FigureWidget(data=data, layout=GO_LAYOUT)
fig_1 = fig_1.update_layout(
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text='local delivery',
        )
    ),
)

In [17]:
# Display graph

# fig_1

### CARD 2

In [18]:
# Format products

df_long = df_past.unstack().reset_index()
df_long = df_long.rename(columns={"level_0": "product", "level_1": "publication_date", 0: "price"})

# Remove #NA points

# df_long = df_long.fillna(value=0)
df_long = df_long[~(df_long["price"] == 0)]

In [19]:
# Format graph

data = []
for product, df_product in df_long.groupby("product"):
    symbol = npta.symbol_to_dict(product)
    if symbol['intraday_delivery'] == 'BASE':
        line_dash='solid'
    else:
        line_dash='dot'
    data.append(    
        go.Scatter(
            x=df_product["publication_date"],
            y=df_product["price"],
            line_dash=line_dash,
            line_color=fixing_colour[symbol['calendar_delivery']],
            name=f"{symbol['intraday_delivery']} {symbol['calendar_delivery']}",
            legendgroup=f"{symbol['calendar_delivery'].lower()} products",
        )
    )

fig_2 = go.FigureWidget(data=data, layout=GO_LAYOUT)
fig_2 = fig_2.update_layout(
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text='publication date',
        )
    ),
)

In [20]:
# Display graph

# fig_2

# DASH COMPONENTS

In [21]:
import jupyter_dash

import dash_dangerously_set_inner_html as html
from dash import dcc

import dash_mdc_neptune as mdc

In [22]:
table = html.DangerouslySetInnerHTML(latest_styler.to_html())
table_section = mdc.Section(
    id="section-table",
    size=3,
    children=table,
    options=[{"title": "Latest Prices"}],
)

extras = {
    "style": {"height": "100%", "width": "100%"},
}
pfc_graph = dcc.Graph(
    id="pfc-graph",
    figure=fig_1,
    **extras
)
ytd_graph = dcc.Graph(
    id="ytd-graph",
    figure=fig_2,
    **extras
)
graph_tab = mdc.Tab(
    id="tab-graph",
    children=[pfc_graph, ytd_graph],
    options=[
        {"label": "Forward Prices"},
        {"label": "Historical Prices"},
    ],
)
graph_section = mdc.Section(
    id="section-graph",
    children=graph_tab,
    options=[{"title": "Market Watch"}],
)

page = mdc.Page(children=[table_section, graph_section])
navbar = mdc.NavBar(title="Short-Term Markets")

layout = mdc.Dashboard(children=[navbar, page])

In [23]:
from jupyter_dash import JupyterDash


app = JupyterDash(__name__)
app.layout = layout

app.run_server(mode='external', host="0.0.0.0", port=8001)

Dash app running on http://0.0.0.0:8001/
